# Text Classification

LSTM for Time Series Data is [here](https://colab.research.google.com/drive/1uKRCjXnUdxWFoNv298q4dNOJyfWAPHtX?usp=sharing)

Please download the file text_data.csv from https://drive.google.com/file/d/1F3Zsb6zEdu-yKyrHi3RzpCm9g3M6Vmr5/view?usp=sharing

In [ ]:
!gdown --id 1F3Zsb6zEdu-yKyrHi3RzpCm9g3M6Vmr5

In [ ]:
import pandas as pd

df=pd.read_csv('text_data.csv')
df['class'].value_counts()

punpromotion.csv    2302
CH3Thailand.csv     1934
ejan2016.csv         319
Name: class, dtype: int64

In [ ]:
df.head()

,message,class
0,อย่า โกรธ แม่ ขาย 😭😭 ดู ย้อนหลัง ที่นี่,CH3Thailand.csv
1,แอคชั่น เต็ม แถม ซึ้ง ฉาก หวาน ดู เบื้องหลัง ...,CH3Thailand.csv
2,ร้องไห้ ไหว ทด กลายเป็น ทาส เด็ก ดู ย้อนหลัง ...,CH3Thailand.csv
3,รื้อ กระเป๋า เผ่า ส่อง ไอเทม เด็ด ดู ติดตาม ...,CH3Thailand.csv
4,มีความผิด ที่จะ โดน เฆี่ยน ทด ใจเด็ด ดู ย้อน...,CH3Thailand.csv


In [ ]:
df['word_length'] = df['message'].str.split()
df['word_length'] = df['word_length'].str.len()
df.dropna(inplace=True)

In [ ]:
df['word_length'].sort_values(ascending=False)

2574    355.0
4089    337.0
2113    282.0
2021    233.0
1943    219.0
        ...  
4073      2.0
143       2.0
4500      2.0
2400      2.0
2307      2.0
Name: word_length, Length: 4554, dtype: float64

In [ ]:
df.message.values

array([' อย่า โกรธ แม่ ขาย 😭😭 ดู ย้อนหลัง ที่นี่',
       ' แอคชั่น เต็ม แถม ซึ้ง ฉาก หวาน ดู เบื้องหลัง ลิขิต จันทร์ ฟัง นักแสดง เล่าเรื่อง สนุก กอง ถ่าย ติดตาม กอง วิก  ลิขิต จันทร์ วันพรุ่งนี้ เวลา 09.15 น. ',
       ' ร้องไห้ ไหว ทด กลายเป็น ทาส เด็ก ดู ย้อนหลัง ที่นี่', ...,
       '  บัตรเครดิต ส่วนลด ไม่ต้อง เสียใจ ปัน โปร >> ขั้นตอน ปัน โปร <<  รายละเอียด สถานที่ ส่วนลด บัตรเครดิต ยอด ชำระเงิน facebook message >> รายละเอียด แบ่งปัน <<  สถานที่ Siam Paragon Siam Center Siam SQ 1, CentralWorld  เวลา ศุกร์ เสาร์ อาทิตย์ เวลา 11.00 น.  21.00  ชำระ ค่า อาหาร สินค้า ส่วนลด เงินสด ค่าธรรมเนียม แบ่งปัน >> หมายเหตุ <<  แบ่งปัน ส่วนลด การแลก คะแนน  สงวนสิทธิ์ การเปลี่ยนแปลง แบ่งปัน ส่วนลด แจ้งให้ทราบ ล่วงหน้า',
       '  เบื่อ ไหม พลาด ส่วนลด บัตรเครดิต ปัน โปร',
       ' แบ่งปัน ส่วนลด โปรโมชั่น บัตรเครดิต ร้านอาหาร ร้านค้า เพื่อน ติดตาม รายละเอียด ส่วนลด ช่วงเวลา สถานที่ แบ่งปัน รายสัปดาห์ ที่นี่ ค่าธรรมเนียม แบ่งปัน ^^'],
      dtype=object)

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import SpatialDropout1D

MAX_WORDS = 2500
MAX_SEQUENCE_LENGTH = 355
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(df.message.values)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(df.message.values)

In [ ]:
df.message.values[0]

' อย่า โกรธ แม่ ขาย 😭😭 ดู ย้อนหลัง ที่นี่'

In [ ]:
X[0]

[52, 2129, 115, 108, 401, 1, 22, 102]

In [ ]:
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
Y = pd.get_dummies(df['class']).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
58/58 [==============================] - 105s 2s/step - loss: 0.4391 - accuracy: 0.8777 - val_loss: 0.0832 - val_accuracy: 0.9854
Epoch 2/5
58/58 [==============================] - 96s 2s/step - loss: 0.0553 - accuracy: 0.9832 - val_loss: 0.0444 - val_accuracy: 0.9854
Epoch 3/5
58/58 [==============================] - 90s 2s/step - loss: 0.0938 - accuracy: 0.9751 - val_loss: 0.0516 - val_accuracy: 0.9829
Epoch 4/5
58/58 [==============================] - 89s 2s/step - loss: 0.0294 - accuracy: 0.9908 - val_loss: 0.0410 - val_accuracy: 0.9902
Epoch 5/5
58/58 [==============================] - 89s 2s/step - loss: 0.0170 - accuracy: 0.9957 - val_loss: 0.0411 - val_accuracy: 0.9902


In [ ]:
acc = model.evaluate(X_test, Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(acc[0], acc[1]))

15/15 [==============================] - 2s 120ms/step - loss: 0.0611 - accuracy: 0.9781
Test set
  Loss: 0.061
  Accuracy: 0.978


In [ ]:
posts = ['เชิญ ดู ละคร ย้อนหลัง วันนี้ ทุ่ม', 'อย่า พลาด อาหาร เด็ด ราคา โดน']
X_new_test = tokenizer.texts_to_sequences(posts)
X_new_test = pad_sequences(X_new_test, maxlen=MAX_SEQUENCE_LENGTH)
results = model.predict(X_new_test)

1/1 [==============================] - 1s 638ms/step


In [ ]:
results

array([[9.9806744e-01, 8.2780462e-04, 1.1047603e-03],
       [1.7661239e-01, 1.4656284e-01, 6.7682475e-01]], dtype=float32)

In [ ]:
df['class'].unique()

array(['CH3Thailand.csv', 'ejan2016.csv', 'punpromotion.csv'],
      dtype=object)

# Word Prediction

In [ ]:
import pandas as pd

df = pd.read_csv('text_data.csv')
df.dropna(inplace=True)
df['word_length'] = df['message'].str.split()
df['word_length'] = df['word_length'].str.len()
df = df[df['word_length'] > 3]
df = df[df['class'] == 'CH3Thailand.csv']

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.utils import to_categorical
import numpy as np

MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 355
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')
tokenizer.fit_on_texts(df.message.values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(df.message.values)

X_train = []
Y_train = []

for x in X:
    for i in range(len(x) - 3):
        X_train.append(x[i:i + 3])
        Y_train.append(x[i + 3])

X_train = np.array(X_train)
Y_train = to_categorical(Y_train, num_classes=MAX_WORDS)

In [ ]:
X_train

array([[ 114,  615,   34],
       [ 615,   34,  616],
       [  34,  616,   71],
       ...,
       [   3,   26,    2],
       [  26,    2,   17],
       [   2,   17, 1585]])

In [ ]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(MAX_WORDS, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 50
batch_size = 100

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/50
309/309 [==============================] - 23s 63ms/step - loss: 7.2951 - accuracy: 0.0376 - val_loss: 7.6792 - val_accuracy: 0.0330
Epoch 2/50
309/309 [==============================] - 9s 30ms/step - loss: 6.3830 - accuracy: 0.0479 - val_loss: 7.6688 - val_accuracy: 0.0330
Epoch 3/50
309/309 [==============================] - 7s 24ms/step - loss: 6.2624 - accuracy: 0.0479 - val_loss: 7.7175 - val_accuracy: 0.0330
Epoch 4/50
309/309 [==============================] - 8s 27ms/step - loss: 6.1855 - accuracy: 0.0479 - val_loss: 7.7724 - val_accuracy: 0.0330
Epoch 5/50
309/309 [==============================] - 8s 26ms/step - loss: 6.0956 - accuracy: 0.0479 - val_loss: 7.8291 - val_accuracy: 0.0330
Epoch 6/50
309/309 [==============================] - 7s 24ms/step - loss: 6.0058 - accuracy: 0.0478 - val_loss: 7.8625 - val_accuracy: 0.0327
Epoch 7/50
309/309 [==============================] - 8s 27ms/step - loss: 5.9216 - accuracy: 0.0472 - val_loss: 7.9264 - val_accuracy: 0.033

In [ ]:
def gen_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text

    for _ in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # pred_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        predict_x = model.predict(pad_encoded, verbose=0)
        pred_word_ind = np.argmax(predict_x, axis=1)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' ' + pred_word
        output_text.append(pred_word)

    return ' '.join(output_text)

In [ ]:
df['message']

0                 อย่า โกรธ แม่ ขาย 😭😭 ดู ย้อนหลัง ที่นี่
1        แอคชั่น เต็ม แถม ซึ้ง ฉาก หวาน ดู เบื้องหลัง ...
2        ร้องไห้ ไหว ทด กลายเป็น ทาส เด็ก ดู ย้อนหลัง ...
3        รื้อ กระเป๋า เผ่า ส่อง  ไอเทม เด็ด ดู ติดตาม ...
4        มีความผิด ที่จะ โดน เฆี่ยน ทด ใจเด็ด  ดู ย้อน...
                              ...                        
1929     พรุ่งนี้ แน่ ตอนแรก สนุก กก กก ชม ดู  เรื่อง ...
1930     ยุบ พอง หนอง ยุบ พอง ซิก แพ แน่น อา ธีร เดช  ...
1931                                      โย กาฝาก ดู ตอน
1932         แนะนำ นักแสดง เรื่อง โน่ ชา  ชา โน่ เล่น เออ
1933      ได้เวลา ตามหา รักแท้  ทันตแพทย์ หนุ่ม หล่อ ร...
Name: message, Length: 1930, dtype: object

In [ ]:
seed_text = "แอคชั่น มีความผิด โดน"
output = gen_text(model, tokenizer, seq_len=3, seed_text=seed_text, num_gen_words=10)
print(f'Output: {seed_text} {output}')

Output: แอคชั่น มีความผิด โดน สาธารณะ แอปพลิเคชัน ละคร ย้อนหลัง 62 ep ลิ้งค์ รัก ที่นี่ ตอนแรก
